In [1]:
#DATA PREPROCESSING

import json

data = json.load(open('en_de_corpus.json', 'r'))

#Script to remove punctuation, ensure <= max allowed length + padding

import string
import random
import math

#replace punctuation with space, ensure all sentences with  1 space
#also all capital to small 
def replacePunctuation(x):
    puncAndNum = string.punctuation+'01233456789'
    x = x.lower()
    for char in x:
        if char in puncAndNum:
            #print(char)
            x=x.replace(char,' ')
    return ' '.join(x.split())

#max allowed raw length=5, pad to make it 5, allow only equal pairs
tempEn = list(map(replacePunctuation,data['en']))
tempDe = list(map(replacePunctuation,data['de']))
maxRawLength = 5
#maxFinalLength = maxRawLength + 2

def wordCount(x):
    return len(x.split())

data_en_processed = list()
data_de_processed = list()
for i in range(len(tempEn)):
    if(wordCount(tempEn[i]) <= maxRawLength and wordCount(tempDe[i]) <= maxRawLength
       and wordCount(tempEn[i])==wordCount(tempDe[i])):
        #if(i%10==0):
        #    print((maxFinalLength -1 - wordCount(tempEn[i])))
        data_en_processed.append(tempEn[i])
        data_de_processed.append(tempDe[i])

print("length after processing : "+str(len(data_en_processed)))  

length after processing : 21680


In [2]:
data_en_processed[0]

'i took the bus back'

In [3]:
#pick 5 random samples and display, compare with google translate
import time
from mtranslate import translate


start = math.ceil(random.random()*1000)
for iiii in range(start,start+2):
    print(data_en_processed[iiii]+" | "+data_de_processed[iiii])
    print(translate(data_en_processed[iiii],'de','en'))
    print()


the cheese is yellow | der käse ist gelb
Der Käse ist gelb

that baby has charming eyes | dieses baby hat hübsche augen
Das Baby hat reizende Augen



In [4]:
# Convert to list of lists from list of strings, remove space

data_en_processed_tokens = [s.split() for s in data_en_processed]
data_de_processed_tokens = [s.split() for s in data_de_processed]

#Lower the size
data_en_processed_tokens = data_en_processed_tokens#[:10] #sentence count
data_de_processed_tokens = data_de_processed_tokens#[:10]


#data_full = data_en_processed_tokens + data_de_processed_tokens
#len(data_full)

#data_full = sum(data_full, [])
#print(len(data_full))

#print(data_full[0:5])
#print(data_full[(len(data_full)//2)+0 :(len(data_full)//2)+5])


print(data_en_processed_tokens[-1])
print(data_de_processed_tokens[-1])

['tom', 'goes', 'to', 'bed', 'early']
['tom', 'geht', 'früh', 'ins', 'bett']


In [5]:
#Partition back into En and De

#data_en =  data_full[:(len(data_full)//2)]
#data_de =  data_full[(len(data_full)//2):]
#print(len(data_en))
#print(len(data_de))

In [6]:
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


#modelEn = gensim.models.Word2Vec(tempEnList,size=100,workers=4,min_count=1)
modelEn = gensim.models.Word2Vec(data_en_processed_tokens,size=100,workers=6,min_count=1)
modelEn.save('english_word2vec_1sentence')

#modelDe = gensim.models.Word2Vec(tempDeList,size=100,workers=4,min_count=1)
modelDe = gensim.models.Word2Vec(data_de_processed_tokens,size=100,workers=6,min_count=1)
modelDe.save('german_word2vec_1sentence')

#print(tempEnList[0])
#print(tempDeList[0])

#tempEnList
#modelEn['lunch']

2017-04-21 01:19:36,135 : INFO : collecting all words and their counts
2017-04-21 01:19:36,137 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-21 01:19:36,138 : INFO : collected 42 word types from a corpus of 47 raw words and 10 sentences
2017-04-21 01:19:36,139 : INFO : Loading a fresh vocabulary
2017-04-21 01:19:36,141 : INFO : min_count=1 retains 42 unique words (100% of original 42, drops 0)
2017-04-21 01:19:36,142 : INFO : min_count=1 leaves 47 word corpus (100% of original 47, drops 0)
2017-04-21 01:19:36,143 : INFO : deleting the raw counts dictionary of 42 items
2017-04-21 01:19:36,145 : INFO : sample=0.001 downsamples 42 most-common words
2017-04-21 01:19:36,146 : INFO : downsampling leaves estimated 11 word corpus (24.5% of prior 47)
2017-04-21 01:19:36,147 : INFO : estimated required memory for 42 words and 100 dimensions: 54600 bytes
2017-04-21 01:19:36,149 : INFO : resetting layer weights
2017-04-21 01:19:36,151 : INFO : training model wi

In [37]:
#modelEn = gensim.models.Word2Vec.load('english_word2vec_100')
#modelDe = gensim.models.Word2Vec.load('german_word2vec_100')

In [16]:
modelEn .most_similar('bus')

[('your', 0.16837053000926971),
 ('tom', 0.16373106837272644),
 ('chinese', 0.1621803641319275),
 ('would', 0.15444834530353546),
 ('the', 0.14593392610549927),
 ('pass', 0.12450592964887619),
 ('study', 0.09771684557199478),
 ('believe', 0.08773599565029144),
 ('thanks', 0.08179964870214462),
 ('back', 0.0805492177605629)]

In [11]:
data_en = sum(data_en_processed_tokens, [])
data_de = sum(data_de_processed_tokens, [])
print(len(data_en))
print(len(data_de))

47
47


In [13]:
import numpy as np
from mtranslate import translate
from pprint import pprint



chars_en = list(set(data_en))
chars_de = list(set(data_de))
vocab_size = 100

print('data has %d en_chars, %d de_chars, %d vec size.' % (len(data_en), len(data_de), vocab_size))

char_en_to_vec = { ch:modelEn[ch] for ch in chars_en }
char_de_to_vec = { ch:modelDe[ch] for ch in chars_de }


hidden_size = 50 # size of hidden layer of neurons
seq_length = 5 # number of steps to unroll the RNN for
learning_rate = 1e-2 #1e-1

#globals
#en_ch_cnt = (len(data_full)//2) #len(data_full)//2
test_sentence = ['i', 'took', 'the', 'bus', 'back']
#test_sentence = ['<', 'he', 'has', 'blond', 'hair', '>', '>']
test_sentence_vec = [char_en_to_vec[ch] for ch in test_sentence]
#'<', 'ich', 'nahm', 'den', 'bus', 'zurück', '>'
test_sentence_de= translate(' '.join(test_sentence),'de', 'en').split()

# model parameters




conc_size = hidden_size + vocab_size

#Input gate
Wi = np.random.randn(hidden_size, conc_size)*0.01 # conc input to hidden
bi = np.zeros((hidden_size, 1))

#Forget gate
Wf = np.random.randn(hidden_size, conc_size)*0.01 # conc input to hidden
bf = np.zeros((hidden_size, 1))


#Output gate
Wo = np.random.randn(hidden_size, conc_size)*0.01 # conc input to hidden
bo = np.zeros((hidden_size, 1))


#Cell State
Wc = np.random.randn(hidden_size, conc_size)*0.01 # conc input to hidden
bc = np.zeros((hidden_size, 1))

Wy = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output

bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev, cprev):

  conc_inp = {}
  xs, hs, ys = {}, {}, {}
  hs_gi, hs_gf, hs_go, hs_ce = {}, {}, {}, {}
  cs = {}

  hs[-1] = np.copy(hprev)
  cs[-1] = np.copy(cprev)
    
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    
    xs[t] = np.reshape(np.array(inputs[t]), (vocab_size,1) )
    conc_inp[t] = np.reshape( np.hstack( ( hs[t-1].flatten(), xs[t].flatten()) ) , (conc_size,1) )    
    hs_gi[t] = np.tanh(np.dot(Wi, conc_inp[t]) + bi)
    hs_gf[t] = np.tanh(np.dot(Wf, conc_inp[t]) + bf)
    hs_go[t] = np.tanh(np.dot(Wo, conc_inp[t]) + bo)
    hs_ce[t] = np.tanh(np.dot(Wc, conc_inp[t]) + bc)
    
    cs[t] = np.multiply(hs_gf[t], cs[t-1]) + np.multiply(hs_gi[t], hs_ce[t])
    
    hs[t] = hs_go[t] * np.tanh(cs[t]) # hidden state

    ys[t] = np.dot(Wy, hs[t]) + by 


    
    loss += 0.5*np.square(np.subtract(np.reshape(np.array(targets[t]),(vocab_size,1)) , ys[t]))
    
  # backward pass: compute gradients going backwards

  dWi, dWf, dWo, dWc, dWy = np.zeros_like(Wi), np.zeros_like(Wf), np.zeros_like(Wo), np.zeros_like(Wc), np.zeros_like(Wy)

  dbi, dbf, dbo, dbc, dby = np.zeros_like(bi), np.zeros_like(bf), np.zeros_like(bo), np.zeros_like(bc), np.zeros_like(by)

  dhnext = np.zeros_like(hs[0])
  dcnext = np.zeros_like(cs[0])  
    
  for t in reversed(range(len(inputs))):
    dy = -1*np.subtract(np.reshape(np.array(targets[t]),(vocab_size,1)) , ys[t])
    dWy += np.dot(dy, hs[t].T)
    dby += dy
    
    dh = np.dot(Wy.T, dy) + dhnext # backprop into h
    
    dho = (1 - hs_go[t] * hs_go[t]) * np.tanh(cs[t]) * dh
    dc = (hs_go[t] * dh * (1 - cs[t] * cs[t])) + dcnext
    dhf = (1 - hs_gf[t] * hs_gf[t]) * cs[t-1] * dc
    dhi = (1 - hs_gi[t] * hs_gi[t]) * hs_ce[t] * dc    
    dhc = (1 - hs_ce[t] * hs_ce[t]) * hs_gi[t] * dc
    
    #print(dhf.shape)
    
    dWf += np.dot(dhf, conc_inp[t].T)
    dbf += dhf
    dXf = np.dot(Wf.T, dhf)
    
    dWi += np.dot(dhi, conc_inp[t].T)
    dbi += dhi
    dXi = np.dot(Wi.T, dhi)
    
    dWo += np.dot(dho, conc_inp[t].T)
    dbo += dho
    dXo = np.dot(Wo.T, dho)
    
    
    dWc += np.dot(dhc, conc_inp[t].T)
    dbc += dhc
    dXc = np.dot(Wc.T, dhc)
    #print(dhnext.shape)
    
    dX = dXo + dXc + dXi + dXf
    
    dhnext = dX[:hidden_size]
    dcnext = hs_gf[t] * dc
    
    
  for dparam in [dWi, dWf, dWo, dWc, dWy, dbi, dbf, dbo, dbc, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWi, dWf, dWo, dWc, dWy, dbi, dbf, dbo, dbc, dby, hs[len(inputs)-1], cs[len(inputs)-1]


def sample(h, c, n):

  n = len(test_sentence_vec)

  translated_vecs = []
  for t in range(n):
    
    #xs[t] = np.reshape(np.array(inputs[t]), (vocab_size,1) )
    x = np.reshape(np.array(test_sentence_vec[t]), (vocab_size,1) )
    
    #LSTM Modifications
    
    
    conc_inp = np.reshape( np.hstack( ( h.flatten(), x.flatten()) ) , (conc_size,1) )    
    h_gi = np.tanh(np.dot(Wi, conc_inp) + bi)
    h_gf = np.tanh(np.dot(Wf, conc_inp) + bf)
    h_go = np.tanh(np.dot(Wo, conc_inp) + bo)
    h_ce = np.tanh(np.dot(Wc, conc_inp) + bc)
    
    c = np.multiply(h_gf, c) + np.multiply(h_gi, h_ce)
    
    h = h_go * np.tanh(c) # hidden state

    y = np.dot(Wy, h) + by 
        
    
    translated_vecs.append(y)
  return translated_vecs






n, p = 0, 0
mWi, mWf, mWo, mWc, mWy = np.zeros_like(Wi), np.zeros_like(Wf), np.zeros_like(Wo), np.zeros_like(Wc), np.zeros_like(Wy)
mbi, mbf, mbo, mbc, mby = np.zeros_like(bi), np.zeros_like(bf),np.zeros_like(bo), np.zeros_like(bc), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data_en) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    cprev = np.zeros((hidden_size,1)) # Reset Cell State
    
    p = 0 # go from start of data
    
  inputs = [char_en_to_vec[ch] for ch in data_en[p:p+seq_length]]
  targets = [char_de_to_vec[ch] for ch in data_de[p:p+seq_length]]

  # sample from the model now and then
  if n % 3000 == 0:
    slen = len(test_sentence_vec)
    sample_vecs = sample(hprev, cprev, slen)
    ans = [modelDe.similar_by_vector(v.flatten(), topn=3, restrict_vocab=None) for v in sample_vecs]
    pprint(ans)

    


  # forward seq_length characters through the net and fetch gradient
  loss, dWi, dWf, dWo, dWc, dWy, dbi, dbf, dbo, dbc, dby, hprev, cprev = lossFun(inputs, targets, hprev, cprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print('iter %d, loss: %f' % (n, np.mean(smooth_loss)) ) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wi,  Wf,  Wo,   Wc,  Wy,  bi,  bf,  bo,  bc,  by], 
                                [dWi, dWf, dWo, dWc, dWy, dbi, dbf, dbo, dbc, dby], 
                                [mWi, mWf, mWo, mWc, mWy, mbi, mbf, mbo, mbc, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


data has 47 en_chars, 47 de_chars, 100 vec size.


2017-04-21 01:22:47,168 : INFO : precomputing L2-norms of word weight vectors


[[('dieses', 0.14226718246936798),
  ('war', 0.12881416082382202),
  ('der', 0.09571588039398193)],
 [('bus', 0.189137265086174),
  ('reich', 0.1828273981809616),
  ('geht', 0.1710958480834961)],
 [('der', 0.21723151206970215),
  ('in', 0.19692270457744598),
  ('schwer', 0.19470684230327606)],
 [('früh', 0.1849265992641449),
  ('haare', 0.17277801036834717),
  ('ins', 0.1467362940311432)],
 [('danke', 0.2852460443973541),
  ('trägt', 0.2427685260772705),
  ('früh', 0.18597164750099182)]]
iter 0, loss: 23.002825
iter 1000, loss: 8.458047
iter 2000, loss: 3.109998
[[('ich', 0.5139691829681396),
  ('würden', 0.2672577500343323),
  ('in', 0.25831010937690735)],
 [('ich', 0.5139691829681396),
  ('würden', 0.2672577500343323),
  ('in', 0.25831010937690735)],
 [('ich', 0.5139691829681396),
  ('würden', 0.2672577500343323),
  ('in', 0.25831007957458496)],
 [('ich', 0.5139691829681396),
  ('würden', 0.2672577500343323),
  ('in', 0.25831010937690735)],
 [('ich', 0.5139691829681396),
  ('würden',

KeyboardInterrupt: 

In [40]:
print(data_en)

['<', 'i', 'took', 'the', 'bus', 'back', '>', '<', 'without', 'air', 'we', 'would', 'die', '>', '<', 'i', 'study', 'chinese', 'in', 'beijing', '>', '<', 'pass', 'me', 'the', 'salt', 'please', '>', '<', 'the', 'boy', 'is', 'wearing', 'glasses', '>', '<', 'we', 'know', 'this', 'song', '>', '>', '<', 'that', 'was', 'hard', 'to', 'believe', '>', '<', 'he', 'has', 'blond', 'hair', '>', '>', '<', 'thanks', 'for', 'your', 'patience', '>', '>', '<', 'tom', 'goes', 'to', 'bed', 'early', '>']


In [ ]:
    print('---------------')
    print('Input Sentence : '+' '.join(test_sentence))
    print('Expected Translation by Google : '+' '.join(test_sentence_de))
    
    ctri = 0
    for v in sample_vecs:
        top_n = modelDe.similar_by_vector(v.flatten(), topn=3, restrict_vocab=None)
        print("word "+str(ctri+1)+" : ",end='')
        for j in range(3):
            print(top_n[j][0],"%.2f" %top_n[j][1] ," , " ,end=' ')
        ctri=ctri+1
        print()
    if(n==40000):
        break

In [13]:
import numpy as np
from mtranslate import translate
from pprint import pprint
import pickle


# data I/O
#data = open('input_en_de_1000.txt', 'r').read() # should be simple plain text file
#data = data_full
chars_en = list(set(data_en))
chars_de = list(set(data_de))
vocab_size = 100

print('data has %d en_chars, %d de_chars, %d vec size.' % (len(data_en), len(data_de), vocab_size))

char_en_to_vec = { ch:modelEn[ch] for ch in chars_en }
char_de_to_vec = { ch:modelDe[ch] for ch in chars_de }

#vec_en_to_char = {  }
#vec_de_to_char = {  }

#char_to_ix = { ch:i for i,ch in enumerate(chars) }
#ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters---> 43     loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)

hidden_size = 250 # size of hidden layer of neurons
seq_length = 7 # number of steps to unroll the RNN for
learning_rate = 1e-2 #1e-1

#globals
#en_ch_cnt = (len(data_full)//2) #len(data_full)//2
test_sentence = ['<', 'i', 'took', 'the', 'bus', 'back', '>']
#test_sentence = [ '<', 'the', 'boy', 'is', 'wearing', 'glasses', '>']
#test_sentence = [ '<', 'i', 'ate', 'a', 'delicious', 'apple', '>']
test_sentence_de= translate(' '.join(test_sentence),'de', 'en').split()

test_sentence_vec = [char_en_to_vec[ch] for ch in test_sentence]
#'<', 'ich', 'nahm', 'den', 'bus', 'zurück', '>'

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias
modelParametersDict = {}

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    #xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    #xs[t][inputs[t]] = 1
    xs[t] = np.reshape(np.array(inputs[t]), (vocab_size,1) )
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    #ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    

    #loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    loss += 0.5*np.square(np.subtract(np.reshape(np.array(targets[t]),(vocab_size,1)) , ys[t]))
    
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #dy = np.copy(ps[t])
    #dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dy = -1*np.subtract(np.reshape(np.array(targets[t]),(vocab_size,1)) , ys[t])
    
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


def sample(h, n):

  n = len(test_sentence_vec)

  translated_vecs = []
  for t in range(n):
    #x = np.zeros((vocab_size, 1))
    #x[char_to_ix[test_sentence[t]]] = 1
    x = np.reshape(np.array(test_sentence_vec[t]), (vocab_size,1) )
        
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by   
    translated_vecs.append(y)
  return translated_vecs






n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data_en) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_en_to_vec[ch] for ch in data_en[p:p+seq_length]]
  #targets = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_de_to_vec[ch] for ch in data_de[p:p+seq_length]]

  # sample from the model now and then
  if n % 10000 == 0:
    slen = len(test_sentence_vec)
    sample_vecs = sample(hprev, slen)
    
    print('---------------')
    print('Input Sentence : '+' '.join(test_sentence))
    print('Expected Translation by Google : '+' '.join(test_sentence_de))
    
    i = 0
    for v in sample_vecs:
        top_n = modelDe.similar_by_vector(v.flatten(), topn=3, restrict_vocab=None)
        print("word "+str(i+1)+" : ",end='')
        for j in range(3):
            print(top_n[j][0],"%.2f" %top_n[j][1] ," , " ,end=' ')
        i=i+1
        print()
    if(n==40000):
        break

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print('iter %d, loss: %f' % (n, np.mean(smooth_loss)) ) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


data has 700 en_chars, 700 de_chars, 100 vec size.


2017-04-20 23:47:22,468 : INFO : precomputing L2-norms of word weight vectors


---------------
Input Sentence : < i took the bus back >
Expected Translation by Google : <Ich nahm den Bus zurück>
word 1 : diese 0.26  ,  musik 0.26  ,  für 0.20  ,  
word 2 : vögel 0.31  ,  normal 0.26  ,  mittagessen 0.22  ,  
word 3 : schön 0.27  ,  liebe 0.23  ,  > 0.22  ,  
word 4 : > 0.23  ,  in 0.23  ,  alleine 0.23  ,  
word 5 : lernen 0.30  ,  dieses 0.27  ,  karthago 0.23  ,  
word 6 : ich 0.28  ,  unnütz 0.27  ,  würdest 0.23  ,  
word 7 : zucker 0.24  ,  mag 0.23  ,  vögel 0.21  ,  
iter 0, loss: 32.203955
iter 1000, loss: 11.841275
iter 2000, loss: 4.353994
iter 3000, loss: 1.600955
iter 4000, loss: 0.588675
iter 5000, loss: 0.216464
iter 6000, loss: 0.079603
iter 7000, loss: 0.029280
iter 8000, loss: 0.010776
iter 9000, loss: 0.003972
---------------
Input Sentence : < i took the bus back >
Expected Translation by Google : <Ich nahm den Bus zurück>
word 1 : < 1.00  ,  sie 0.42  ,  > 0.41  ,  
word 2 : ich 0.97  ,  ist 0.38  ,  tom 0.37  ,  
word 3 : nahm 0.46  ,  ist 0.

In [31]:
# test_sentence_vec, hprev, 

print(sample_vecs)

modelParametersDict['Wxh'] = Wxh
modelParametersDict['Whh'] = Whh
modelParametersDict['Why'] = Why
modelParametersDict['bh'] = bh
modelParametersDict['by'] = by
modelParametersDict['hprev'] = hprev
modelParametersDict['char_en_to_vec'] = char_en_to_vec
with open('modelParameters.pickle', 'wb') as handle:
    print("saved model at iteration outside loop")
    pickle.dump(modelParametersDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        
slen = len(test_sentence_vec)
sample_vecs = sample(hprev, slen)

print('---------------')
print('Input Sentence : '+' '.join(test_sentence))
print('Expected Translation by Google : '+' '.join(test_sentence_de))

i = 0
for v in sample_vecs:
    top_n = modelDe.similar_by_vector(v.flatten(), topn=3, restrict_vocab=None)
    print("word "+str(i+1)+" : ",end='')
    for j in range(3):
        print(top_n[j][0],"%.2f" %top_n[j][1] ," , " ,end=' ')
    i=i+1
    print()
#modelDe.similar_by_vector(yy, topn=1, restrict_vocab=None) 

[array([[  2.07670734e-03],
       [ -4.06371152e-04],
       [ -3.42080426e-04],
       [  5.15222125e-03],
       [  1.84234927e-03],
       [ -2.08671396e-04],
       [ -4.25859257e-04],
       [ -5.63789543e-03],
       [ -1.80087852e-03],
       [ -3.57092576e-03],
       [ -3.60609454e-04],
       [ -1.20417018e-03],
       [ -2.11774490e-03],
       [ -1.88793261e-03],
       [ -2.86027079e-04],
       [  5.10478535e-04],
       [ -3.80731930e-03],
       [ -6.12193420e-04],
       [  2.33198220e-03],
       [ -1.26030728e-03],
       [  2.41871383e-03],
       [  3.50779070e-03],
       [  3.33497706e-03],
       [  5.50822940e-03],
       [  2.07445445e-03],
       [  7.92457594e-05],
       [ -9.84727590e-04],
       [ -1.92324555e-03],
       [  2.54320804e-03],
       [  5.20021176e-04],
       [  2.96060715e-03],
       [ -8.77065247e-04],
       [ -2.61733737e-03],
       [  5.13997292e-04],
       [ -1.98663648e-03],
       [  1.57545716e-03],
       [  3.06609369e-03],


In [15]:
len(inputs)

7

In [16]:
chars_de

['lerne',
 'als',
 'großvater',
 'kein',
 'mary',
 'willkommen',
 'diese',
 'duftet',
 'vorbei',
 'sterben',
 'unterrichtet',
 'solltest',
 'verletzen',
 '<',
 'bitte',
 'telefon',
 'es',
 'muss',
 'füße',
 'hoffnung',
 'zucker',
 'normal',
 'beschreiben',
 'borg',
 'ich',
 'wir',
 'und',
 'apfel',
 '>',
 'alleine',
 'hat',
 'sehr',
 'karl',
 'schwarz',
 'glauben',
 'könnten',
 'nahm',
 'jetzt',
 'früh',
 'abschied',
 'willst',
 'gewesen',
 'wann',
 'bei',
 'wurde',
 'würdest',
 'fast',
 'höre',
 'drei',
 'beeilen',
 'wo',
 'keine',
 'geht',
 'niederländische',
 'brille',
 'habe',
 'eine',
 'nichts',
 'zurück',
 'schnell',
 'fliegen',
 'roman',
 'hatten',
 'schwöre',
 'schmidt',
 'schön',
 'verloren',
 'gott',
 'begann',
 'steht',
 'ohne',
 'roch',
 'machte',
 'hält',
 'für',
 'zugelassen',
 'blume',
 'spreche',
 'bus',
 'alt',
 'weinen',
 'etwas',
 'reise',
 'macht',
 'sofort',
 'eifersüchtig',
 'vögel',
 'schulde',
 'radio',
 'ab',
 'kannst',
 'mich',
 'mittag',
 'da',
 'mach',
 'wär

In [17]:
sample_vecs[0].flatten().shape

(100,)

In [18]:
test_sentence = ['i', 'i', 'took', 'the', 'bus', 'back', 'i']
test_sentence_vec = [char_en_to_vec[ch] for ch in test_sentence]


In [19]:
%whos

Variable                   Type              Data/Info
------------------------------------------------------
Whh                        ndarray           250x250: 62500 elems, type `float64`, 500000 bytes (488.28125 kb)
Why                        ndarray           100x250: 25000 elems, type `float64`, 200000 bytes (195.3125 kb)
Wxh                        ndarray           250x100: 25000 elems, type `float64`, 200000 bytes (195.3125 kb)
bh                         ndarray           250x1: 250 elems, type `float64`, 2000 bytes
by                         ndarray           100x1: 100 elems, type `float64`, 800 bytes
char_de_to_vec             dict              n=267
char_en_to_vec             dict              n=247
chars_de                   list              n=267
chars_en                   list              n=247
dWhh                       ndarray           250x250: 62500 elems, type `float64`, 500000 bytes (488.28125 kb)
dWhy                       ndarray           100x250: 25000 elems

In [20]:
Variable                   Type        Data/Info
------------------------------------------------
Whh                        ndarray     50x50: 2500 elems, type `float64`, 20000 bytes
Why                        ndarray     497x50: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
Wxh                        ndarray     50x497: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
bh                         ndarray     50x1: 50 elems, type `float64`, 400 bytes
by                         ndarray     497x1: 497 elems, type `float64`, 3976 bytes
char_to_ix                 dict        n=497
chars                      list        n=497
dWhh                       ndarray     50x50: 2500 elems, type `float64`, 20000 bytes
dWhy                       ndarray     497x50: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
dWxh                       ndarray     50x497: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
data                       list        n=1400
data_de_processed          list        n=21680
data_de_processed_tokens   list        n=100
data_en_processed          list        n=21680
data_en_processed_tokens   list        n=100
data_full                  list        n=1400
data_size                  int         1400
dbh                        ndarray     50x1: 50 elems, type `float64`, 400 bytes
dby                        ndarray     497x1: 497 elems, type `float64`, 3976 bytes
dparam                     ndarray     497x1: 497 elems, type `float64`, 3976 bytes
en_ch_cnt                  int         700
gensim                     module      <module 'gensim' from '/h<...>ages/gensim/__init__.py'>
hidden_size                int         50
hprev                      ndarray     50x1: 50 elems, type `float64`, 400 bytes
i                          int         259
inputs                     list        n=7
ix_to_char                 dict        n=497
json                       module      <module 'json' from '/hom<...>hon3.5/json/__init__.py'>
learning_rate              float       0.01
loss                       float64     23.793629885
lossFun                    function    <function lossFun at 0x7f61930aeb70>
mWhh                       ndarray     50x50: 2500 elems, type `float64`, 20000 bytes
mWhy                       ndarray     497x50: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
mWxh                       ndarray     50x497: 24850 elems, type `float64`, 198800 bytes (194.140625 kb)
math                       module      <module 'math' from '/hom<...>3.5/lib-dynload/math.so'>
maxFinalLength             int         7
maxRawLength               int         5
mbh                        ndarray     50x1: 50 elems, type `float64`, 400 bytes
mby                        ndarray     497x1: 497 elems, type `float64`, 3976 bytes
mem                        ndarray     497x1: 497 elems, type `float64`, 3976 bytes
modelEn                    Word2Vec    Word2Vec(vocab=6972, size=100, alpha=0.025)
modelGe                    Word2Vec    Word2Vec(vocab=9279, size=100, alpha=0.025)
n                          int         2459
np                         module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
p                          int         581
param                      ndarray     497x1: 497 elems, type `float64`, 3976 bytes
random                     module      <module 'random' from '/h<...>lib/python3.5/random.py'>
replacePunctuation         function    <function replacePunctuation at 0x7f61b38527b8>
sample                     function    <function sample at 0x7f61c4e2b0d0>
sample_ix                  list        n=7
seq_length                 int         7
slen                       int         7
smooth_loss                float64     21.9345595918
start                      int         255
string                     module      <module 'string' from '/h<...>lib/python3.5/string.py'>
targets                    list        n=7
tempDe                     list        n=132173
tempEn                     list        n=132173
test_sentence              list        n=7
time                       module      <module 'time' (built-in)>
translate                  function    <function translate at 0x7f61b27228c8>
txt                        str         < ich ist ist > > >
vocab_size                 int         497
wordCount                  function    <function wordCount at 0x7f61b2b3d510>
x                          ndarray     100: 100 elems, type `float32`, 400 bytes
y                          ndarray     100: 100 elems, type `float32`, 400 bytes

SyntaxError: invalid syntax (<ipython-input-20-11e3c2eba05c>, line 1)